# Lab 3 - Nim

### Task 3.1 - An agent using fixed rules based on nim-sum
Based on the explanation available here: https://en.wikipedia.org/wiki/Nim  

It wants to finish every move with a nim-sum of 0, called 'secure position' (then it will win if it does not make mistakes). 

In [1]:
import logging
import random
from copy import deepcopy

In [2]:
logging.basicConfig(format="%(message)s", level=logging.INFO)

In [3]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i*2 + 1 for i in range(num_rows)]
        self._k = k

    def nimming(self, row: int, num_objects: int) -> None:
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects
        if sum(self._rows) == 0:
            logging.info("\n*** Game over! *** \n")

In [4]:
def nim_sum(elem: list):
  x = 0
  for e in elem:
    x = e ^ x
  return x

## Implementation

In [5]:
def expert_action(board, k):
  """
    The agent uses fixed rules based on nim-sum (expert-system)

    Returns the index of the pile and the number of pieces removed
  """
  # Winning move if there is only one row with more than 1 obj
  # If there is only one obj left, the current player lost
  tmp = [(i, r) for i, r in enumerate(board) if r > 0]
  if len(tmp) == 1:
    row, num_obj = tmp[0]
    if num_obj == 1:
      return row, num_obj
    elif k and num_obj > k:
      return row, k
    else:
      return row, num_obj-1 #left to 1


  # Compute the nim-sum of all the heap sizes
  x = 0
  for r in board:
    x = r ^ x
  assert x >= 0

  if x > 0:
    # Current player on a insucure position -> is winning
    # --> Has to generate a secure position (bad for the other player)
    # --> Find a heap where the nim-sum of X and the heap-size is less than the heap-size.
    # --> Then play on that heap, reducing the heap to the nim-sum of its original size with X
    
    good_rows = [] # A list is needed because of k
    for row, row_size in enumerate(board):
      if row_size == 0:
        continue
      ns = row_size ^ x # nim sum
      if ns < row_size:
        good_rows.append((row, row_size)) # This row will have nim sum = 0
        
    for row, row_size in good_rows:
      board_tmp = deepcopy(board)
      for i in range(row_size):
       board_tmp[row] -= 1 
       if nim_sum(board_tmp) == 0:  # winning move
        num_obj = abs(board[row] - board_tmp[row]) # <----- DA SISTEMARE -------
        if not k or num_obj <= k:
          return row, num_obj
  
  # x == 0 or k force a bad move to the player
  # Current player on a secure position or on a bad position bc of k -> is losing
  # --> Can only generate an insicure position (good for the other player)
  not_zero_rows = [(i, row_size) for i, row_size in enumerate(board) if board[i] > 0]
  row, num_obj = random.choice(not_zero_rows)
  if k and num_obj > k:
    return row, 1 # Take just one obj from the row
  else:
    return row, num_obj # Clear the entire row

## Play a game

In [6]:
def play_nim(num_rows, k=None, action=expert_action):

  nim = Nim(num_rows, k)
  player_action = {
    0: action, # our champion
    1: expert_action # our opponent
    }
  player = 1
  cnt = 0

  logging.debug(f'Board:\t{nim._rows}')
  logging.debug(f'\tk = {nim._k}')

  while not sum(nim._rows) == 0:
    player = int(not player) 
    row, num_obj = player_action[player](nim._rows, nim._k)
    nim.nimming(row, num_obj)
    cnt += 1
 
    logging.debug(f'player {player} -> takes {num_obj} obj from row {row}')
    logging.debug(f'\t--> {nim._rows}\tnim-sum = {nim_sum(nim._rows)}') 

  logging.info(f'### Player {player} lost in {cnt} moves ###')



Choose the game parameters and play

In [7]:
num_rows = 5
k = None
action = expert_action

logging.getLogger().setLevel(logging.DEBUG)

In [8]:
play_nim(num_rows, k, action)

Board:	[1, 3, 5, 7, 9]
	k = None
player 0 -> takes 9 obj from row 4
	--> [1, 3, 5, 7, 0]	nim-sum = 0
player 1 -> takes 7 obj from row 3
	--> [1, 3, 5, 0, 0]	nim-sum = 7
player 0 -> takes 3 obj from row 2
	--> [1, 3, 2, 0, 0]	nim-sum = 0
player 1 -> takes 1 obj from row 0
	--> [0, 3, 2, 0, 0]	nim-sum = 1
player 0 -> takes 1 obj from row 1
	--> [0, 2, 2, 0, 0]	nim-sum = 0
player 1 -> takes 2 obj from row 1
	--> [0, 0, 2, 0, 0]	nim-sum = 2
player 0 -> takes 1 obj from row 2
	--> [0, 0, 1, 0, 0]	nim-sum = 1
*** Game over! *** 

player 1 -> takes 1 obj from row 2
	--> [0, 0, 0, 0, 0]	nim-sum = 0
### Player 1 lost in 8 moves ###
